In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

D:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# 载入数据集
mnist = input_data.read_data_sets("MNIST_data", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [5]:
# 每个批次的大小
batch_size = 100
# 计算一共有多少个批次
n_batch = mnist.train.num_examples // batch_size

# 初始化权值
def weight_varible(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

# 初始化偏置
def bias_varible(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

# 卷积层
def conv2d(x, W):
    #x input tensor of shape '[batch,in_height,in_width,in_channles]'
    #W filter / kernel tensor of shape [filter_height,filter_width,in_channels,out_channels]
    #`strides[0] = strides[3] = 1`. strides[1]代表x方向的步长，strides[2]代表y方向的步长
    #padding: A `string` from: `"SAME", "VALID"`
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

# 池化层
def max_pool_2x2(x):
    # ksize [1,x,y,1]
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

# 定义两个placeholder
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])

# 改变x的格式转换为4D的向量[batch, in_height, in_width, in_channels]
x_image = tf.reshape(x, [-1, 28, 28, 1])

# 初始化第一个卷积层的权重和偏置
W_conv1 = weight_varible([5, 5, 1, 32])  # 5*5采样窗口，32个卷积核从一个平面抽取特征
b_conv1 = bias_varible([32])  # 每一个卷积核一个偏置值

# 把 x_image 和权重向量进行卷积，再加上偏置值，然后应用于 relu 激活函数
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

# 初始化第二个卷积层的权重和偏置
W_conv2 = weight_varible([5, 5, 32, 64])  # 5*5采样窗口，64个卷积核从32个平面抽取特征
b_conv2 = bias_varible([64])

# 把 h_pool1 和权重向量进行卷积，再加上偏置值，然后应用于 relu 激活函数
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

# 28*28 的图片第一次卷积后还是 28*28， 第一次池化后变为 14*14
# 第二次卷积后为 14*14，池化后变为 7*7
# 经过上面操作后得到 64 张 7*7 的平面

# 初始化第一个全连接层的权值
W_fc1 = weight_varible([7*7*64, 1024])  # 生一层有 7*7*64 个神经元，全连接层有1024个神经元
b_fc1 = bias_varible([1024])

# 把池化层2的输出扁平化为1维
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
# 求第一个全连接层的输出
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# keep_prob 用于表示神经元输出概率
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# 初始化第二个全连接层的权值
W_fc2 = weight_varible([1024, 10])  # 生一层有 7*7*64 个神经元，全连接层有1024个神经元
b_fc2 = bias_varible([10])
prediction = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

# 交叉熵代价函数
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=prediction))
# 使用 AdamOptimizer 进行优化
train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)
# 结果存放在一个布尔型列表中
correct_predition = tf.equal(tf.argmax(y, 1), tf.argmax(prediction, 1))   # argmax 返回一维张量中最大值所在的位置
# 求准确率
accuracy = tf.reduce_mean(tf.cast(correct_predition, tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(21):
        for batch in range(n_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step, feed_dict={x: batch_xs, y: batch_ys, keep_prob: 0.7})
            
        acc = sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels, keep_prob: 1.0})
        print('Iter' + str(epoch) + ', Testing Accuracy ' + str(acc))


Iter0, Testing Accuracy 0.8563
Iter1, Testing Accuracy 0.872
Iter2, Testing Accuracy 0.9703
Iter3, Testing Accuracy 0.9797
Iter4, Testing Accuracy 0.9819
Iter5, Testing Accuracy 0.9853
Iter6, Testing Accuracy 0.9836
Iter7, Testing Accuracy 0.9881
Iter8, Testing Accuracy 0.9887
Iter9, Testing Accuracy 0.9889
Iter10, Testing Accuracy 0.9886
Iter11, Testing Accuracy 0.9905
Iter12, Testing Accuracy 0.9894
Iter13, Testing Accuracy 0.9901
Iter14, Testing Accuracy 0.9911
Iter15, Testing Accuracy 0.9902
Iter16, Testing Accuracy 0.9909
Iter17, Testing Accuracy 0.9911
Iter18, Testing Accuracy 0.9906
Iter19, Testing Accuracy 0.9911
Iter20, Testing Accuracy 0.9911
